In [1]:
import multiprocess as mp
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
import gc

In [2]:
basedir = './국토교통부_건축물대장_총괄표제부/'
filenames = [f for f in os.listdir(basedir) if f.endswith('.txt')]
filenames

['MART_DJY_02_201412.txt',
 'mart_djy_02_201512.txt',
 'mart_djy_02_201612.txt',
 'mart_djy_02_201712.txt',
 'mart_djy_02_201812.txt',
 'mart_djy_02_201912.txt',
 'mart_djy_02_202011.txt']

In [3]:
colnames_df = pd.read_excel(basedir + '데이터구조.xls', header=1)
print(colnames_df.shape)
colnames_df.head()

(64, 3)


,컬럼 한글명,데이터 타입,비고
0,관리_건축물대장_PK,VARCHAR(33),NaN
1,대장_구분_코드,VARCHAR(1),NaN
2,대장_구분_코드_명,VARCHAR(100),NaN
3,대장_종류_코드,VARCHAR(1),NaN
4,대장_종류_코드_명,VARCHAR(100),NaN


In [4]:
colnames = colnames_df['컬럼 한글명'].tolist()
print(len(colnames))
colnames[:5]

64


['관리_건축물대장_PK', '대장_구분_코드', '대장_구분_코드_명', '대장_종류_코드', '대장_종류_코드_명']

In [5]:
dfs_list = []
for i, file in enumerate(filenames):
    yearmonth = int(file.replace('.txt', '')[-6:])
    
    if i == 0:
        df = pd.read_csv(basedir + file, encoding='euc-kr', sep='|')
        colnames_2014 = df.columns.tolist()
        colnames_2014.remove('양성화_여부')
        colnames_2014.remove('특이사항')
        colnames_2014.append('양성화_여부')
        colnames_2014.append('특이사항')
        df.columns = colnames_2014
        df.drop(columns=['양성화_여부', '특이사항'], inplace=True)
        
    else:
        df = pd.read_csv(basedir + file, encoding='euc-kr', sep='|', header=None, names=colnames)
        
    df['기준년월'] = yearmonth
    
    dfs_list.append(df)

c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (16,17,30,47,48,49) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (16,17,30,47,48,49,50,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (16,17,47,48,49,50,51,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
c:\users\max6296\ml\lib\site-packages\IPython\core\interactiveshell.py:3147: DtypeWarning: Columns (15,16,17,47,48,49,50,51,56) have mixed types. Specify dtype option on import or set low_memory=False.
  interactiv

In [6]:
for df in dfs_list:
    print(df.shape)

(535169, 65)
(546192, 65)
(554892, 65)
(563589, 65)
(572676, 65)
(581894, 65)
(588611, 65)


In [7]:
second_cols = dfs_list[1].columns

In [8]:
dfs_list[0].columns = second_cols

In [9]:
first_cols = dfs_list[0].columns

In [15]:
prac_df = dfs_list[0].copy()

In [16]:
prac_df['관리_건축물대장_PK'] = prac_df['관리_건축물대장_PK'].apply(lambda x: x[:-2])

In [17]:
dfs_list[0] = prac_df.copy()
del prac_df
gc.collect()

60

In [18]:
concat_df = pd.concat(dfs_list)
print(concat_df.shape)
concat_df.head()

(3943023, 65)


,관리_건축물대장_PK,대장_구분_코드,대장_구분_코드_명,대장_종류_코드,대장_종류_코드_명,신_구_대장_구분_코드,신_구_대장_구분_코드_명,대지_위치,도로명_대지_위치,건물_명,시군구_코드,법정동_코드,대지_구분_코드,번,지,특수지_명,블록,로트,외필지_수,새주소_도로_코드,새주소_법정동_코드,새주소_지상지하_코드,새주소_본_번,새주소_부_번,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도_코드,주_용도_코드_명,기타_용도,세대_수(세대),가구_수(가구),주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관_코드,허가번호_기관_코드_명,허가번호_구분_코드,허가번호_구분_코드_명,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월
0,29110-1001830,2,집합,1,총괄표제부,1.0,신대장,광주광역시 동구 지산동 671번지,NaN,지산휴먼시아,29110,11100,0,671,0.0,NaN,NaN,NaN,0,2.911043e+11,11102.0,0.0,20.0,0.0,21866.4,3959.29,18.11,48700.17,41857.51,191.42,02000,공동주택,"공동주택(아파트), 근린생활시설",375,0,7,9,6716.33,385,0,0.0,0,0.0,214,5575.94,171,2026.5,2.00507e+07,20080215,2.01006e+07,NaN,NaN,,NaN,,4,NaN,0.0,0,NaN,0,NaN,0,20100622,201412
1,45111-17,2,집합,1,총괄표제부,0.0,구대장,전라북도 전주시 완산구 평화동1가 570-15번지,NaN,극동평화아파트,45111,13200,0,570,15.0,NaN,NaN,NaN,0,NaN,NaN,0.0,NaN,NaN,0.0,0.00,0.00,16372.48,0.00,0.00,02000,공동주택,공동주택,0,0,0,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,NaN,NaN,,NaN,,0,NaN,0.0,0,NaN,0,NaN,0,20100619,201412
2,45130-1001920,1,일반,1,총괄표제부,1.0,신대장,전라북도 군산시 서수면 마룡리 93-16번지,NaN,NaN,45130,34024,0,93,16.0,NaN,NaN,NaN,0,4.513046e+11,34001.0,0.0,66.0,0.0,11550.0,4880.55,42.25,5025.80,5025.80,43.51,17000,공장,공장,0,0,3,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,1991,NaN,,NaN,,0,NaN,0.0,0,NaN,0,NaN,0,20100619,201412
3,45130-1001932,1,일반,1,총괄표제부,1.0,신대장,전라북도 군산시 오식도동 882번지,NaN,(주)대진테크닉,45130,14700,0,882,0.0,NaN,NaN,NaN,0,4.513033e+11,14701.0,0.0,46.0,0.0,9974.9,4623.22,46.35,4877.28,4877.28,48.90,17000,공장,공장,0,0,2,0,0.00,17,0,0.0,0,0.0,0,0.00,17,195.5,2.01002e+07,20100227,2.01006e+07,2010,6.45108e+06,개발지원부,1101.0,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20100619,201412
4,28110-1001734,2,집합,1,총괄표제부,1.0,신대장,인천광역시 중구 을왕동 878-2번지,NaN,썬앤문하우스,28110,14900,0,878,2.0,NaN,NaN,NaN,3,2.811042e+11,14901.0,0.0,25.0,0.0,1829.0,365.20,19.97,1307.68,1307.68,71.50,02000,공동주택,다세대주택,16,0,2,0,0.00,16,0,0.0,0,0.0,0,0.00,16,189.0,2.00508e+07,20070801,2.00807e+07,NaN,NaN,,NaN,,0,NaN,0.0,0,NaN,0,NaN,0,20090317,201412


In [19]:
def correct_addr(x):
    # x is a string
    if pd.isna(x) == True:
        return x
    else:
        x = x.replace('번지', '')
        if x[0] == ' ':
            for i in range(len(x)):
                if x[i] != ' ':
                    x = x[i:]
                    break
        
        splitted = x.split('-')
        if len(splitted) == 1:
            return x+'-0'
        else:
            return x

In [20]:
concat_df['지번주소'] = concat_df['대지_위치'].apply(correct_addr)

In [21]:
concat_df.drop(columns=[col for col in concat_df.columns if col.endswith('_코드')], inplace=True)

In [22]:
concat_df.columns = [col.replace('_코드_명', '') for col in concat_df.columns]
print(concat_df.shape)
concat_df.head()

(3943023, 54)


,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,대지_위치,도로명_대지_위치,건물_명,번,지,특수지_명,블록,로트,외필지_수,새주소_본_번,새주소_부_번,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,세대_수(세대),가구_수(가구),주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,허가번호_기관,허가번호_구분,호_수(호),에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소
0,29110-1001830,집합,총괄표제부,신대장,광주광역시 동구 지산동 671번지,NaN,지산휴먼시아,671,0.0,NaN,NaN,NaN,0,20.0,0.0,21866.4,3959.29,18.11,48700.17,41857.51,191.42,공동주택,"공동주택(아파트), 근린생활시설",375,0,7,9,6716.33,385,0,0.0,0,0.0,214,5575.94,171,2026.5,2.00507e+07,20080215,2.01006e+07,NaN,,,4,NaN,0.0,0,NaN,0,NaN,0,20100622,201412,광주광역시 동구 지산동 671-0
1,45111-17,집합,총괄표제부,구대장,전라북도 전주시 완산구 평화동1가 570-15번지,NaN,극동평화아파트,570,15.0,NaN,NaN,NaN,0,NaN,NaN,0.0,0.00,0.00,16372.48,0.00,0.00,공동주택,공동주택,0,0,0,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,NaN,,,0,NaN,0.0,0,NaN,0,NaN,0,20100619,201412,전라북도 전주시 완산구 평화동1가 570-15
2,45130-1001920,일반,총괄표제부,신대장,전라북도 군산시 서수면 마룡리 93-16번지,NaN,NaN,93,16.0,NaN,NaN,NaN,0,66.0,0.0,11550.0,4880.55,42.25,5025.80,5025.80,43.51,공장,공장,0,0,3,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,1991,,,0,NaN,0.0,0,NaN,0,NaN,0,20100619,201412,전라북도 군산시 서수면 마룡리 93-16
3,45130-1001932,일반,총괄표제부,신대장,전라북도 군산시 오식도동 882번지,NaN,(주)대진테크닉,882,0.0,NaN,NaN,NaN,0,46.0,0.0,9974.9,4623.22,46.35,4877.28,4877.28,48.90,공장,공장,0,0,2,0,0.00,17,0,0.0,0,0.0,0,0.00,17,195.5,2.01002e+07,20100227,2.01006e+07,2010,개발지원부,신축허가,0,NaN,0.0,0,NaN,0,NaN,0,20100619,201412,전라북도 군산시 오식도동 882-0
4,28110-1001734,집합,총괄표제부,신대장,인천광역시 중구 을왕동 878-2번지,NaN,썬앤문하우스,878,2.0,NaN,NaN,NaN,3,25.0,0.0,1829.0,365.20,19.97,1307.68,1307.68,71.50,공동주택,다세대주택,16,0,2,0,0.00,16,0,0.0,0,0.0,0,0.00,16,189.0,2.00508e+07,20070801,2.00807e+07,NaN,,,0,NaN,0.0,0,NaN,0,NaN,0,20090317,201412,인천광역시 중구 을왕동 878-2


In [23]:
concat_df.drop(columns=['대지_위치', '도로명_대지_위치', '번', '지', '특수지_명', '블록', '로트', '새주소_본_번',
                        '새주소_부_번', '허가번호_기관', '허가번호_구분'
                       ], inplace=True)

In [24]:
concat_df['총세대수'] = concat_df['세대_수(세대)'] + concat_df['가구_수(가구)'] + concat_df['호_수(호)']
concat_df.drop(columns=['세대_수(세대)', '가구_수(가구)', '호_수(호)'], inplace=True)
print(concat_df.shape)
concat_df.head()

(3943023, 41)


,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,건물_명,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수
0,29110-1001830,집합,총괄표제부,신대장,지산휴먼시아,0,21866.4,3959.29,18.11,48700.17,41857.51,191.42,공동주택,"공동주택(아파트), 근린생활시설",7,9,6716.33,385,0,0.0,0,0.0,214,5575.94,171,2026.5,2.00507e+07,20080215,2.01006e+07,NaN,NaN,0.0,0,NaN,0,NaN,0,20100622,201412,광주광역시 동구 지산동 671-0,379
1,45111-17,집합,총괄표제부,구대장,극동평화아파트,0,0.0,0.00,0.00,16372.48,0.00,0.00,공동주택,공동주택,0,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,0,NaN,0,20100619,201412,전라북도 전주시 완산구 평화동1가 570-15,0
2,45130-1001920,일반,총괄표제부,신대장,NaN,0,11550.0,4880.55,42.25,5025.80,5025.80,43.51,공장,공장,3,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,1991,NaN,0.0,0,NaN,0,NaN,0,20100619,201412,전라북도 군산시 서수면 마룡리 93-16,0
3,45130-1001932,일반,총괄표제부,신대장,(주)대진테크닉,0,9974.9,4623.22,46.35,4877.28,4877.28,48.90,공장,공장,2,0,0.00,17,0,0.0,0,0.0,0,0.00,17,195.5,2.01002e+07,20100227,2.01006e+07,2010,NaN,0.0,0,NaN,0,NaN,0,20100619,201412,전라북도 군산시 오식도동 882-0,0
4,28110-1001734,집합,총괄표제부,신대장,썬앤문하우스,3,1829.0,365.20,19.97,1307.68,1307.68,71.50,공동주택,다세대주택,2,0,0.00,16,0,0.0,0,0.0,0,0.00,16,189.0,2.00508e+07,20070801,2.00807e+07,NaN,NaN,0.0,0,NaN,0,NaN,0,20090317,201412,인천광역시 중구 을왕동 878-2,16


In [25]:
seoul_df = concat_df[concat_df['지번주소'].str.contains('서울특별시')].reset_index(drop=True)
print(seoul_df.shape)
seoul_df.head()

(144387, 41)


,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,건물_명,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수
0,11650-2,집합,총괄표제부,구대장,NaN,0,359.3,162.78,45.30,493.41,325.56,90.61,공동주택,공동주택,2,0,0.00,2,0,0.0,0,0.0,0,0.00,2,30.0,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 방배동 911-29,8
1,11650-2,집합,총괄표제부,신대장,트라움하우스 5,0,4321.4,1855.67,42.94,12660.24,6129.15,141.83,공동주택,공동주택(연립주택),3,0,0.00,0,0,0.0,0,0.0,110,4522.93,0,0.0,1.99909e+07,19991025,2.00304e+07,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 서초동 1496-26,18
2,11650-2,일반,총괄표제부,신대장,NaN,0,0.0,0.00,0.00,239.73,0.00,0.00,단독주택,"주택, 점포",3,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 방배동 938-24,1
3,11650-2,일반,총괄표제부,구대장,NaN,0,227.9,106.13,46.57,121.73,121.73,53.41,창고시설,"창고시설, 제2종근린생활시설",2,0,0.00,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 잠원동 32-9,0
4,11650-3,일반,총괄표제부,신대장,NaN,0,0.0,0.00,0.00,964.68,0.00,0.00,문화및집회시설,종교시설,3,1,17.26,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 반포동 709-5,2


In [26]:
seoul_df['addr_split_len'] = seoul_df['지번주소'].apply(lambda x: len(x.split(' ')))

In [27]:
seoul_df['addr_split_len'].value_counts()

4    140778
5      3437
6       121
3        33
7        18
Name: addr_split_len, dtype: int64

In [28]:
len3_df = seoul_df[seoul_df['addr_split_len'] == 3]
seoul_df.drop(len3_df.index, inplace=True)
len7_df = seoul_df[seoul_df['addr_split_len'] == 7]
seoul_df.drop(len7_df.index, inplace=True)
len6_df = seoul_df[seoul_df['addr_split_len'] == 6]
seoul_df.drop(len6_df.index, inplace=True)

In [29]:
len5_df = seoul_df[seoul_df['addr_split_len'] == 5]

In [30]:
len5_df['fourth_addr'] = len5_df['지번주소'].apply(lambda x: x.split(' ')[3])

In [31]:
seoul_df['대장구분명'] = np.nan

In [32]:
def landtype_name(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        if x == '산':
            return '산'
        elif ('블록' in x)&('지구' in x):
            return '지구블록'
        elif '블록' in x:
            return 'BL'

In [33]:
len5_df['대장구분명'] = len5_df['fourth_addr'].apply(landtype_name)

In [34]:
seoul_df.loc[len5_df.index, '대장구분명'] = len5_df['대장구분명']

In [35]:
def weird_addr(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        try:
            int(x.split(' ')[-1].replace('-', ''))
            return 0
        except:
            return 1

In [36]:
len4_df = seoul_df[seoul_df['addr_split_len'] == 4]
len4_df['weird_addr'] = len4_df['지번주소'].apply(weird_addr)

In [37]:
weird_addr_df = len4_df[len4_df['weird_addr'] == 1]

In [38]:
bdinfo_df = seoul_df
del seoul_df

In [39]:
bdinfo_df.drop(weird_addr_df.index, inplace=True)

In [40]:
bdinfo_df = bdinfo_df[bdinfo_df['대장구분명'] != '지구블록']

In [41]:
bdinfo_df = bdinfo_df[bdinfo_df['대장구분명'] != 'BL']

In [42]:
bdinfo_df['대장구분명'].fillna('일반', inplace=True)
bdinfo_df['대장구분명'].value_counts()

일반    140699
산       3387
Name: 대장구분명, dtype: int64

In [43]:
bdinfo_df.drop(columns=['addr_split_len'], inplace=True)

In [44]:
bdinfo_san_df = bdinfo_df[bdinfo_df['대장구분명'] == '산']

In [45]:
def san_new_addr(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        splitted = x.split(' ')
        return splitted[0] + ' ' + splitted[1] + ' ' + splitted[2] + ' ' + splitted[3] + splitted[4]

In [46]:
bdinfo_san_df['지번주소'] = bdinfo_san_df['지번주소'].apply(san_new_addr)

In [47]:
bdinfo_df.loc[bdinfo_san_df.index, '지번주소'] = bdinfo_san_df['지번주소']

In [48]:
def addr_split_len(x):
    # x is a string
    
    if pd.isna(x) == True:
        return x
    else:
        return len(x.split(' '))

In [49]:
main_data_df = pd.read_csv('./prepped_data/main_data_baseline_checkpoint_2.csv')

In [50]:
inter_df = bdinfo_df[bdinfo_df['지번주소'].isin(main_data_df['지번주소'].unique())]
print(inter_df.shape)
inter_df.head()

(44480, 42)


,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,건물_명,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),허가_일,착공_일,사용승인_일,허가번호_년,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명
0,11650-2,집합,총괄표제부,구대장,NaN,0,359.3,162.78,45.30,493.41,325.56,90.61,공동주택,공동주택,2,0,0.0,2,0,0.0,0,0.0,0,0.00,2,30.00,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 방배동 911-29,8,일반
1,11650-2,집합,총괄표제부,신대장,트라움하우스 5,0,4321.4,1855.67,42.94,12660.24,6129.15,141.83,공동주택,공동주택(연립주택),3,0,0.0,0,0,0.0,0,0.0,110,4522.93,0,0.00,1.99909e+07,19991025,2.00304e+07,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 서초동 1496-26,18,일반
6,11650-3,집합,총괄표제부,신대장,방배동 갤러리하우스,1,884.0,413.10,46.73,3695.03,2346.09,265.39,공동주택,공동주택(아파트),2,0,0.0,28,0,0.0,0,0.0,28,1305.56,0,0.00,2.00201e+07,200204,2.00305e+07,2002,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 방배동 1-7,16,일반
7,11650-4,집합,총괄표제부,구대장,NaN,1,380.5,190.20,49.98,570.60,380.40,99.97,공동주택,공동주택,2,0,0.0,3,0,0.0,0,0.0,0,0.00,3,41.25,NaN,NaN,NaN,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 반포동 577-73,12,일반
8,11650-,집합,총괄표제부,신대장,방배동다세대주택,0,493.0,292.30,59.29,1234.89,1095.26,222.16,공동주택,"공동주택,다세대주택",2,0,0.0,12,0,0.0,0,0.0,12,0.00,0,0.00,2.00205e+07,20030325,2.00309e+07,2002,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 서초구 방배동 919-10,15,일반


In [51]:
inter_df.drop(columns=['허가_일', '착공_일', '허가번호_년'], inplace=True)

In [52]:
def is_intable(x):
    
    if pd.isna(x) == True:
        return x
    else:
        try:
            int(x)
            return 1
        except:
            return 0

In [53]:
inter_df['사용승인일_intable'] = inter_df['사용승인_일'].apply(is_intable)

In [54]:
inter_df['사용승인일_intable'].value_counts()

1.0    23371
Name: 사용승인일_intable, dtype: int64

In [55]:
def apply_int(x):
    if pd.isna(x) == True:
        return x
    else:
        return int(x)

In [56]:
inter_df['사용승인_일'] = inter_df['사용승인_일'].apply(apply_int)

In [57]:
inter_df['사용승인_일'] = inter_df['사용승인_일'].astype('Int64')

In [58]:
inter_df.drop(columns=['사용승인일_intable'], inplace=True)

In [59]:
inter_df['사용승인_일'].describe()

count    2.337100e+04
mean     1.999839e+07
std      1.274720e+06
min      1.994020e+05
25%      2.003120e+07
50%      2.010093e+07
75%      2.015071e+07
max      2.032033e+07
Name: 사용승인_일, dtype: float64

In [60]:
inv_df = inter_df[inter_df['사용승인_일'] == inter_df['사용승인_일'].max()]

In [61]:
addr_df = inter_df[inter_df['지번주소'] == inv_df['지번주소'].iloc[0]]

In [62]:
inter_df.loc[inv_df.index, '사용승인_일'] = 20020328

In [63]:
inter_df['사용승인_일'].describe()

count    2.337100e+04
mean     1.999830e+07
std      1.274708e+06
min      1.994020e+05
25%      2.003120e+07
50%      2.010091e+07
75%      2.015071e+07
max      2.020112e+07
Name: 사용승인_일, dtype: float64

In [64]:
inter_df['기준년월_최초사용승인일'] = inter_df[['지번주소', '기준년월', '사용승인_일']].groupby(['지번주소', '기준년월']).transform('min')['사용승인_일']
inter_df['기준년월_최근사용승인일'] = inter_df[['지번주소', '기준년월', '사용승인_일']].groupby(['지번주소', '기준년월']).transform('max')['사용승인_일']

In [65]:
inter_df['주소별_최초생성일자'] = inter_df[['지번주소', '생성_일자']].groupby(['지번주소']).transform('min')['생성_일자']
inter_df['주소별_최근생성일자'] = inter_df[['지번주소', '생성_일자']].groupby(['지번주소']).transform('max')['생성_일자']

In [66]:
inter_df['주소별_기준년월_총세대수합'] = inter_df[['지번주소', '기준년월', '총세대수']].groupby(['지번주소', '기준년월']).transform('sum')['총세대수']
inter_df['주소별_기준년월_총세대수합'].describe()

count    44480.000000
mean       233.740310
std        488.231225
min          0.000000
25%         16.000000
50%         30.000000
75%        245.000000
max       6864.000000
Name: 주소별_기준년월_총세대수합, dtype: float64

In [67]:
inter_df.shape

(44480, 44)

In [68]:
zero_df = inter_df[inter_df['주소별_기준년월_총세대수합'] == 0]
print(zero_df.shape)

(1031, 44)


In [69]:
zero_df['지번주소'].nunique()

233

In [70]:
count = 0
for addr in zero_df['지번주소'].unique():
    addr_df = inter_df[inter_df['지번주소'] == addr]
    print(addr_df.shape)
    display(addr_df)
    count += 1
    if count == 5:
        break

(6, 44)


,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,건물_명,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,주소별_기준년월_총세대수합
31,11545-6,일반,총괄표제부,구대장,NaN,1,1271.4,712.73,56.05,811.57,712.73,56.05,공장,공장,2,1,19.83,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 금천구 가산동 234-2,0,일반,NaN,NaN,20100417,20200416,0
21494,11545-666,일반,총괄표제부,구대장,NaN,1,1271.4,712.73,56.05,811.57,712.73,56.05,공장,공장,2,1,19.83,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201512,서울특별시 금천구 가산동 234-2,0,일반,NaN,NaN,20100417,20200416,0
42674,11545-666,일반,총괄표제부,구대장,NaN,1,1271.4,712.73,56.05,811.57,712.73,56.05,공장,공장,2,1,19.83,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201612,서울특별시 금천구 가산동 234-2,0,일반,NaN,NaN,20100417,20200416,0
63408,11545-666,일반,총괄표제부,구대장,NaN,1,1271.4,712.73,56.05,811.57,712.73,56.05,공장,공장,2,1,19.83,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201712,서울특별시 금천구 가산동 234-2,0,일반,NaN,NaN,20100417,20200416,0
84079,11545-666,일반,총괄표제부,구대장,NaN,1,1271.4,712.73,56.05,811.57,712.73,56.05,공장,공장,2,1,19.83,0,0,0.0,0,0.0,0,0.00,0,0.0,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201812,서울특별시 금천구 가산동 234-2,0,일반,NaN,NaN,20100417,20200416,0
139576,11545-100251899,집합,총괄표제부,신대장,금강노블레스,1,1269.4,432.12,34.04,4275.52,3801.98,299.51,업무시설,업무시설(오피스텔),2,0,0.00,78,68,222.0,0,0.0,6,259.28,4,46.0,20200413,NaN,0.0,0,NaN,0,NaN,0,20200416,202011,서울특별시 금천구 가산동 234-2,96,일반,20200413,20200413,20100417,20200416,96


(7, 44)


,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,건물_명,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,주소별_기준년월_총세대수합
145,11740-3,일반,총괄표제부,신대장,NaN,0,902.7,517.88,57.37,1125.05,954.67,105.76,제2종근린생활시설,"제1종,제2종근린생활시설,",3,0,0.0,69,0,0.0,0,0.0,0,0.00,6,69.0,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 강동구 천호동 293-17,0,일반,NaN,NaN,20100417,20171205,0
21249,11740-390,일반,총괄표제부,신대장,NaN,0,902.7,517.88,57.37,1125.05,954.67,105.76,제2종근린생활시설,"제1종,제2종근린생활시설,",3,0,0.0,69,0,0.0,0,0.0,0,0.00,6,69.0,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201512,서울특별시 강동구 천호동 293-17,0,일반,NaN,NaN,20100417,20171205,0
43762,11740-100223738,집합,총괄표제부,신대장,중앙클레식하우스,0,902.7,485.29,53.76,2538.53,1801.44,199.56,공동주택,근린생활시설 및 도시형생활주택(단지형다세대),2,0,0.0,25,0,0.0,0,0.0,12,431.85,13,157.4,20160128,NaN,0.0,69,NaN,0,NaN,0,20160206,201612,서울특별시 강동구 천호동 293-17,28,일반,20160128,20160128,20100417,20171205,28
68183,11740-100223738,집합,총괄표제부,신대장,중앙클레식하우스,0,902.7,485.29,53.76,2538.53,1801.44,199.56,공동주택,근린생활시설 및 도시형생활주택(단지형다세대),2,0,0.0,25,0,0.0,0,0.0,12,431.85,13,157.4,20160128,NaN,0.0,69,NaN,0,NaN,0,20171205,201712,서울특별시 강동구 천호동 293-17,28,일반,20160128,20160128,20100417,20171205,28
87462,11740-100223738,집합,총괄표제부,신대장,중앙클레식하우스,0,902.7,485.29,53.76,2538.53,1801.44,199.56,공동주택,근린생활시설 및 도시형생활주택(단지형다세대),2,0,0.0,25,0,0.0,0,0.0,12,431.85,13,157.4,20160128,NaN,0.0,69,NaN,0,NaN,0,20171205,201812,서울특별시 강동구 천호동 293-17,28,일반,20160128,20160128,20100417,20171205,28
107608,11740-100223738,집합,총괄표제부,신대장,중앙클레식하우스,0,902.7,485.29,53.76,2538.53,1801.44,199.56,공동주택,근린생활시설 및 도시형생활주택(단지형다세대),2,0,0.0,25,0,0.0,0,0.0,12,431.85,13,157.4,20160128,NaN,0.0,69,NaN,0,NaN,0,20171205,201912,서울특별시 강동구 천호동 293-17,28,일반,20160128,20160128,20100417,20171205,28
132952,11740-100223738,집합,총괄표제부,신대장,중앙클레식하우스,0,902.7,485.29,53.76,2538.53,1801.44,199.56,공동주택,근린생활시설 및 도시형생활주택(단지형다세대),2,0,0.0,25,0,0.0,0,0.0,12,431.85,13,157.4,20160128,NaN,0.0,69,NaN,0,NaN,0,20171205,202011,서울특별시 강동구 천호동 293-17,28,일반,20160128,20160128,20100417,20171205,28


(1, 44)


,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,건물_명,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,주소별_기준년월_총세대수합
180,11740-5,일반,총괄표제부,신대장,NaN,0,1012.4,133.2,13.16,199.2,199.2,19.68,제2종근린생활시설,제2종근린생활시설,2,0,0.0,1,0,0.0,0,0.0,0,0.0,1,11.5,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 강동구 암사동 459-1,0,일반,NaN,NaN,20100417,20100417,0


(1, 44)


,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,건물_명,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,주소별_기준년월_총세대수합
253,11710-5,일반,총괄표제부,신대장,NaN,0,495.3,244.98,49.46,274.58,244.98,49.46,제1종근린생활시설,제1종근린생활시설,2,0,0.0,0,0,0.0,0,0.0,0,0.0,0,0.0,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 송파구 방이동 130-0,0,일반,NaN,NaN,20100417,20100417,0


(2, 44)


,관리_건축물대장_PK,대장_구분,대장_종류,신_구_대장_구분,건물_명,외필지_수,대지_면적(㎡),건축_면적(㎡),건폐_율(%),연면적(㎡),용적_률_산정_연면적(㎡),용적_률(%),주_용도,기타_용도,주_건축물_수,부속_건축물_수,부속_건축물_면적(㎡),총_주차_수,옥내_기계식_대수(대),옥내_기계식_면적(㎡),옥외_기계식_대수(대),옥외_기계식_면적(㎡),옥내_자주식_대수(대),옥내_자주식_면적(㎡),옥외_자주식_대수(대),옥외_자주식_면적(㎡),사용승인_일,에너지효율_등급,에너지절감_율,에너지_EPI점수,친환경_건축물_등급,친환경_건축물_인증점수,지능형_건축물_등급,지능형_건축물_인증점수,생성_일자,기준년월,지번주소,총세대수,대장구분명,기준년월_최초사용승인일,기준년월_최근사용승인일,주소별_최초생성일자,주소별_최근생성일자,주소별_기준년월_총세대수합
262,11440-10,일반,총괄표제부,구대장,NaN,0,3206.7,1446.21,45.1,1314.59,1198.39,37.37,자동차관련시설,자동차관련시설,4,0,0.0,9,0,0.0,0,0.0,9,279.21,0,0.0,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201412,서울특별시 마포구 성산동 593-5,0,일반,NaN,NaN,20100417,20100417,0
21436,11440-1050,일반,총괄표제부,구대장,NaN,0,3206.7,1446.21,45.1,1314.59,1198.39,37.37,자동차관련시설,자동차관련시설,4,0,0.0,9,0,0.0,0,0.0,9,279.21,0,0.0,NaN,NaN,0.0,0,NaN,0,NaN,0,20100417,201512,서울특별시 마포구 성산동 593-5,0,일반,NaN,NaN,20100417,20100417,0


In [71]:
inter_df['주_용도'].value_counts()

공동주택          41002
단독주택           1145
업무시설            509
제1종근린생활시설       353
제2종근린생활시설       189
공장              145
                109
노유자시설            51
숙박시설             34
종교시설             33
자동차관련시설          24
창고시설             20
교육연구시설           16
위험물저장및처리시설       16
교육연구및복지시설        10
판매시설              7
문화및집회시설           4
근린생활시설            3
Name: 주_용도, dtype: int64

In [72]:
inter_df.drop(zero_df.index, inplace=True)
inter_df.shape

(43449, 44)

In [75]:
inter_df.to_csv('./국토교통부_건축물대장_총괄표제부/wbdinfo_baseline_2.csv', index=False)